In [1]:
from pymongo import MongoClient
client = MongoClient()
db = client.whosampled
import numpy as np
import pandas as pd
import re

import sys
np.set_printoptions(threshold=sys.maxsize)

from turn_db_main_into_utility_matrix import from_mongo_collection_to_utility_matrix

In [36]:
# Remove (2) from producer name
df['new_song_producer'] = df.new_song_producer.apply(lambda x: re.sub('\(2\)|\(Producer\)', '', x))

In [37]:
prods = df['new_song_producer'].unique()

In [38]:
prods.sort()

In [39]:
prods

array(['"Buttnaked" Tim Dawg', '$uicideboy$', '10', '12 Million',
       "1500 or Nothin'", '2 in Rhythm', '2Pac', '3 Boyz From Newark',
       '30 Roc', '42 North', '45 Music', '4th Disciple', '5th Ward Boyz',
       '6ix', '7 Aurelius', '7L', '8-Off Agallah', '808 Mafia',
       '808&amp;EliTE', '808-Ray', '88-Keys', '8Track', '9th Wonder',
       'A Kid Called Roots', 'A New Beginning', 'A Tribe Called Quest',
       'A$AP Ty Beats', 'A-Trak', 'A. G. Cook', 'AHk 2Gs', 'Aalias',
       'Abraham Orellana', 'Adam Bravin', 'Adam Deitch', 'Adam Pallin',
       'Adrian Younge', 'Aesop Rock', 'Afrika Islam', 'Agallah',
       'Ahmir "Questlove" Thompson', 'Air Smooth', 'Aja Grant',
       'Akhenaton', 'Akon', 'Al "Butter" McLean', 'Al Be Back', 'Al West',
       'Alan Braxe', 'Alex Coletti', 'Alex Gale', 'Alex Medina',
       'Alexander "Spanador" Mosely', 'Ali Shaheed Muhammad',
       'Alias (Rapper)', 'Allen Ritter', 'Allstar', 'Allstar Fresh',
       'Amp', 'AmpLive', 'Analog(ue) Tape 

In [18]:
# Problem
# How many times are we missing data on producers?
df = pd.DataFrame(list(db.main_redo.find()))
df[df.new_song_producer == "None Listed"]

#Very few?
#Only lists 31 times there is no new_song_producer.

,URL,_id,contributor_points,elements_sampled,name_of_contributor,new_song_album,new_song_artist,new_song_name,new_song_producer,new_song_year,"presence_of_""and throughout""_in_description",sampled_artist,sampled_song_album,sampled_song_name,sampled_song_producer,sampled_song_year,time_in_sampled_song_where_sample_appears
16709,https://www.whosampled.com/sample/320821/Kid-I...,5ca634f690a02a1758fb304e,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed,None Listed
16712,https://www.whosampled.com/sample/350888/Killa...,5ca6393890a02a17d5b5a4a4,Contributed by\ndennis1990 (30 ),Direct Sample of Vocals / Lyrics,dennis1990,Lean on Me: The Adventures of Joe Clark,Killa Kyleon,My City,None Listed,2013,Sample appears at 0:24 (and throughout),Jodeci,Forever My Lady,Forever My Lady,"[DeVanté Swing, Al B. Sure!]",1991,0:18
16713,https://www.whosampled.com/sample/350888/Killa...,5ca6395a90a02a1809813b0b,Contributed by\ndennis1990 (30 ),Direct Sample of Vocals / Lyrics,dennis1990,Lean on Me: The Adventures of Joe Clark,Killa Kyleon,My City,None Listed,2013,Sample appears at 0:24 (and throughout),Jodeci,Forever My Lady,Forever My Lady,"[DeVanté Swing, Al B. Sure!]",1991,0:18
16714,https://www.whosampled.com/sample/548231/Kirko...,5ca6396a90a02a1809813b0c,Contributed by\nIndy Steward (40 ),Direct Sample of Multiple Elements,Indy Steward,Playa Made,Kirko Bangz,1:45,None Listed,2016,Sample appears at 0:00 (and throughout),Donell Jones,Where I Wanna Be,Shorty (Got Her Eyes on Me),"[Donell Jones, Eric Williams, Wesley ""Big Wes""...",1999,0:03
16715,https://www.whosampled.com/sample/411721/Kirko...,5ca6397890a02a1809813b0d,Contributed by\njazz1808 (50 ),Direct Sample of Multiple Elements,jazz1808,PlayaMade,Kirko Bangz,Ain't a Pimp,None Listed,2016,Sample appears at 0:02 (and throughout),En Vogue,Born to Sing,Don't Go,"[Denzil Foster, Thomas McElroy]",1990,0:39
16716,https://www.whosampled.com/sample/301583/Kirko...,5ca6398790a02a1809813b0e,"Contributed by\nbvcon (16,473 )",Direct Sample of Multiple Elements,bvcon,Progression 4,Kirko Bangz,Check Me Out,None Listed,2014,Sample appears at 0:00 (and throughout),Nipsey Hussle,Crenshaw,Checc Me Out,[1500 or Nothin'],2013,0:00
16718,https://www.whosampled.com/sample/301585/Kirko...,5ca639a490a02a1809813b10,"Contributed by\nbvcon (16,473 )",Interpolation (Replayed Sample) of Multiple El...,bvcon,Progression 4,Kirko Bangz,Don't Matter to Me,None Listed,2014,Sample appears at 0:00 (and throughout),Justin Bieber,Journals,All That Matters,"[D.K. the Punisher, Poo Bear, Justin Bieber]",2013,0:16
16721,https://www.whosampled.com/sample/263243/Kirko...,5ca639cd90a02a1809813b13,"Contributed by\njbcedeno95 (77,337 )",Direct Sample of Multiple Elements,jbcedeno95,Progression IV,Kirko Bangz,Fuck U,None Listed,2014,Sample appears at 0:00 (and throughout),Yo Gotti,Nov 19th: The Mixtape,Fuck You,[Infamous],2013,0:00
16722,https://www.whosampled.com/sample/140877/Kirko...,5ca639e090a02a1809813b14,Contributed by\nlouey (10 ),Direct Sample of Multiple Elements,louey,Progression 2: A Young Texas Playa,Kirko Bangz,I's a Playa,None Listed,2012,Sample appears at 0:00 (and throughout),Pimp C,The Sweet James Jones Stories,I'sa Playa,[N.O. Joe],2005,0:00
16723,https://www.whosampled.com/sample/185228/Kirko...,5ca639ed90a02a1809813b15,"Contributed by\njbcedeno95 (77,337 )",Direct Sample of Multiple Elements,jbcedeno95,Procrastination Kills,Kirko Bangz,Ice Cream Bang Job,None Listed,2009,Sample appears at 0:03 (and throughout),Dorrough,Dorrough Music,Ice Cream Paint Job,[2Much],2009,0:14


In [2]:
utility_mat = from_mongo_collection_to_utility_matrix(db.main_redo)


There are 15809 unique records. There are 1324 producers. There are 7789 songs sampled.

In [8]:
yo = utility_mat.sum(axis = 0).sort_values()

In [9]:
yo

sampled_song_name
밤의찬가2                                          1
Telephone Line                                 1
Telephone Rock                                 1
Juaacklyn                                      1
Joywind                                        1
Joyful Grass and Grape                         1
Joy Trip                                       1
Tell 'Em                                       1
Tell Her                                       1
Journey to the Light                           1
Tell Me                                        1
Josephina De Granada                           1
Jordan Speaks at a Seminar                     1
Tell Me How Long It's Been                     1
Johnny the Fox Meets Jimmy the Weed            1
Johnny Too Bad                                 1
Telephone Girl                                 1
Juicy Fruit (Fruity Instrumental Mix)          1
Julia                                          1
Julie Is a TV Set (EP Version)                 1
Ju

In [25]:
new.sum(axis = 0)

sampled_song_name
"...3 Minutes 45 Seconds and Counting..." "... Three, Two, One, Zero. All Engine     2
"Dies Irae" Requiem Mass in D Minor II                                               1
"Ripples (Montage) (1966)"                                                           1
"T" Plays It Cool                                                                    3
"What Kind of a Cop Killer Are You !?"                                               1
'93 Swing                                                                            1
'Cause I Love You (1978 Version)                                                     5
'Ole Love                                                                            1
'Til the Cops Come Knockin'                                                          1
'Till I Collapse                                                                     1
'Where All the Good Song Have Gone?' Jingle                                          1
(Don't Ask Me) to Stop Lo

In [7]:
yo = pd.crosstab(utility_mat['new_song_producer'],columns=utility_mat['sampled_artist_and_name']).astype(bool)                                                  

In [8]:
yo

sampled_artist_and_name  0                            James Brown\n1                            James Brown\n2                            James Brown\n3                            Lyn Collins\n4                            Lyn Collins\n5                            Lyn Collins\n6                            James Brown\n7                             Grady Tate\n8                 Booker T. & the M.G.'s\n9                                 Beside\n10                             Kevie Kev\n11                           Stubby Kaye\n12                          The Sequence\n13                          Leroy Hutson\n14                     Maceo & the Macks\n15                Graham Central Station\n16                    Average White Band\n17                       The Rubber Band\n18              Love Unlimited Orchestra\n19                         The Dramatics\n20                                Manzel\n21                            Cilvaringz\n22                       Kool & the Gang\n23                            Peter Nero\n24                         The New Birth\n25                      Frederick Knight\n26                             Al Pacino\n27                          Ohio Players\n28                      The Bible (Book)\n29                                  Lobo\n                      ...               \n16674    The Cecil Holmes Soulful Sounds\n16675                                 YG\n16676                    Kool & the Gang\n16677                    Kool & the Gang\n16678                    Kool & the Gang\n16679                    Eric B. & Rakim\n16680                          Tom Jones\n16681                          Tom Jones\n16682                          Tom Jones\n16683                    Eric B. & Rakim\n16684                          Bob James\n16685                          Bob James\n16686                          Bob James\n16687                       Judas Priest\n16688                      The Artistics\n16689               Kool G Rap & DJ Polo\n16690               Kool G Rap & DJ Polo\n16691               Kool G Rap & DJ Polo\n16692                              Rakim\n16693    Little Royal & the Swingmasters\n16694    Little Royal & the Swingmasters\n16695    Little Royal & the Swingmasters\n16696                             M.O.P.\n16697                             M.O.P.\n16698                   Maynard Ferguson\n16699                   The Fatback Band\n16700                             Rory-O\n16701                            Run-DMC\n16702                        James Brown\n16703                       Jack Trombey\nName: sampled_artist, Length: 16704, dtype: object - 0                                           It's a New Day\n1                                           It's a New Day\n2                                           It's a New Day\n3                        We Want to Parrty, Parrty, Parrty\n4                        We Want to Parrty, Parrty, Parrty\n5                        We Want to Parrty, Parrty, Parrty\n6                        Get Up, Get Into It, Get Involved\n7                                            Be Black Baby\n8                                                 Grab Bag\n9                         Change the Beat (Female Version)\n10                               All Night Long (Waterbed)\n11                                      Fugue for Tinhorns\n12                                             Funk You Up\n13                                        Love the Feeling\n14                                           Soul Power 74\n15                                                 The Jam\n16                                            Cut the Cake\n17                                              Rubber Jam\n18                            Makin' Believe That It's You\n19                                             In the Rain\n20                                          Midnight Theme\n21                              The Epic of Bitter Triumph\n22                                      

In [5]:
new = pd.pivot_table(utility_mat, index = ['new_song_producer'], columns= ['sampled_artist_and_name'])

In [6]:
new.head()

new_song_year
sampled_artist_and_name 0                            James Brown\n1                            James Brown\n2                            James Brown\n3                            Lyn Collins\n4                            Lyn Collins\n5                            Lyn Collins\n6                            James Brown\n7                             Grady Tate\n8                 Booker T. & the M.G.'s\n9                                 Beside\n10                             Kevie Kev\n11                           Stubby Kaye\n12                          The Sequence\n13                          Leroy Hutson\n14                     Maceo & the Macks\n15                Graham Central Station\n16                    Average White Band\n17                       The Rubber Band\n18              Love Unlimited Orchestra\n19                         The Dramatics\n20                                Manzel\n21                            Cilvaringz\n22                       Kool & the Gang\n23                            Peter Nero\n24                         The New Birth\n25                      Frederick Knight\n26                             Al Pacino\n27                          Ohio Players\n28                      The Bible (Book)\n29                                  Lobo\n                      ...               \n16674    The Cecil Holmes Soulful Sounds\n16675                                 YG\n16676                    Kool & the Gang\n16677                    Kool & the Gang\n16678                    Kool & the Gang\n16679                    Eric B. & Rakim\n16680                          Tom Jones\n16681                          Tom Jones\n16682                          Tom Jones\n16683                    Eric B. & Rakim\n16684                          Bob James\n16685                          Bob James\n16686                          Bob James\n16687                       Judas Priest\n16688                      The Artistics\n16689               Kool G Rap & DJ Polo\n16690               Kool G Rap & DJ Polo\n16691               Kool G Rap & DJ Polo\n16692                              Rakim\n16693    Little Royal & the Swingmasters\n16694    Little Royal & the Swingmasters\n16695    Little Royal & the Swingmasters\n16696                             M.O.P.\n16697                             M.O.P.\n16698                   Maynard Ferguson\n16699                   The Fatback Band\n16700                             Rory-O\n16701                            Run-DMC\n16702                        James Brown\n16703                       Jack Trombey\nName: sampled_artist, Length: 16704, dtype: object - 0                                           It's a New Day\n1                                           It's a New Day\n2                                           It's a New Day\n3                        We Want to Parrty, Parrty, Parrty\n4                        We Want to Parrty, Parrty, Parrty\n5                        We Want to Parrty, Parrty, Parrty\n6                        Get Up, Get Into It, Get Involved\n7                                            Be Black Baby\n8                                                 Grab Bag\n9                         Change the Beat (Female Version)\n10                               All Night Long (Waterbed)\n11                                      Fugue for Tinhorns\n12                                             Funk You Up\n13                                        Love the Feeling\n14                                           Soul Power 74\n15                                                 The Jam\n16                                            Cut the Cake\n17                                              Rubber Jam\n18                            Makin' Believe That It's You\n19                                             In the Rain\n20                                          Midnight Theme\n21                              The Epic of Bitter Triumph\n22                         

In [125]:
import scipy.sparse as sparse
data_sparse = sparse.lil_matrix(data)

from ItemItemRecommender import ItemItemRecommender
recommend = ItemItemRecommender()
recommend.fit(data_sparse)


array([1., 0., 0., ..., 1., 1., 0.])

In [127]:
recomenders_for_4th_Disciple = recommend.pred_one_user(11)

In [137]:
ind = np.argwhere(recomenders_for_4th_Disciple)
data.columns[ind]

Index(['It's a New Day', 'Change the Beat (Female Version)', 'Rubber Jam',
       'Makin' Believe That It's You', 'In the Rain', 'Midnight Theme',
       'The Epic of Bitter Triumph', 'N.T.',
       'Love Theme From "Romeo and Juliet" (A Time for Us)', 'Do It Again',
       ...
       'Think Big', 'The Eye of a Needle', 'One Less Bell to Answer',
       'Don't Say Goodnight (It's Time for Love) (Parts 1 & 2)',
       'The Tragedy (Don't Do It)', 'Ain't No Half Steppin'',
       'Kool It (Here Comes the Fuzz)', 'Trilogy of Terror', 'L Dopa',
       'Do It Your Way'],
      dtype='object', length=1011)

In [113]:
# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

KeyError: 'user'

In [138]:
df.head()

,URL,contributor_points,elements_sampled,name_of_contributor,new_song_album,new_song_artist,new_song_name,new_song_producer,new_song_year,"presence_of_""and throughout""_in_description",sampled_artist,sampled_song_album,sampled_song_name,sampled_song_producer,sampled_song_year,time_in_sampled_song_where_sample_appears
0,https://www.whosampled.com/sample/126703/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Hook / Riff,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),The 45 King,1988,Sample appears at 1:24 (and throughout),James Brown,It's a New Day - Let a Man Come In,It's a New Day,James Brown,1970,0:12
1,https://www.whosampled.com/sample/126703/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Hook / Riff,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),Marley Marl,1988,Sample appears at 1:24 (and throughout),James Brown,It's a New Day - Let a Man Come In,It's a New Day,James Brown,1970,0:12
3,https://www.whosampled.com/sample/126696/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Vocals / Lyrics,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),The 45 King,1988,Sample appears at 0:00,Lyn Collins,None Listed,"We Want to Parrty, Parrty, Parrty",James Brown,1973,0:00
4,https://www.whosampled.com/sample/126696/Big-D...,"Contributed by\nDukeOne (44,467 )",Direct Sample of Vocals / Lyrics,DukeOne,12 Inch U.K. Release Only,Big Daddy Kane,Set It Off (Extended Mix),Marley Marl,1988,Sample appears at 0:00,Lyn Collins,None Listed,"We Want to Parrty, Parrty, Parrty",James Brown,1973,0:00
6,https://www.whosampled.com/sample/3499/Big-Dad...,Contributed by\nasymmetry (373 ),Direct Sample of Hook / Riff,asymmetry,Long Live the Kane,Big Daddy Kane,Set It Off,The 45 King,1988,Sample appears at 0:07 (and throughout),James Brown,None Listed,"Get Up, Get Into It, Get Involved",James Brown,1970,3:33


In [139]:
utility2.head()

,It's a New Day,"We Want to Parrty, Parrty, Parrty","Get Up, Get Into It, Get Involved",Be Black Baby,Grab Bag,Change the Beat (Female Version),All Night Long (Waterbed),Fugue for Tinhorns,Funk You Up,Love the Feeling,...,When the Morning Comes,Ain't No Half Steppin',2 Da Good Tymz,Kool It (Here Comes the Fuzz),Bloodstone,Trilogy of Terror,How About Some Hardcore,L Dopa,Do It Your Way,Legless
"""Buttnaked"" Tim Dawg",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
$uicideboy$,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12 Million,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1500 or Nothin',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)


#---------------------
# FIND SIMILAR ITEMS
#---------------------

# Find the 10 most similar to Jay-Z
item_id = 147068 #Jay-Z
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print data.artist.loc[data.artist_id == idx].iloc[0]

    
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 2025
user_id = 2025

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print recommendations
